<a href="https://colab.research.google.com/github/deenukhan/deep_learning/blob/main/3_Loading_and_Pre_processing_in_TF_KERAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Notebook is inspired the book Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow

# In this notebook we will be performing below tasks

# 1. In this exercise you will download a dataset, split it, create a tf.data.Dataset to load it and preprocess 
#    it efficiently, then build and train a binary classification model containing an Embedding layer:

    # Download the Large Movie Review Dataset, which contains 50,000 movies reviews from the Internet Movie Database. 
    # The data is organized in two directories, train and test, each containing a pos subdirectory with 12,500 positive reviews and 
    # a neg subdirectory with 12,500 negative reviews. Each review is stored in a separate text file. There are other files and folders 
    # (including preprocessed bag-of-words), but we will ignore them in this exercise.

    # Split the test set into a validation set (15,000) and a test set (10,000).

    # Use tf.data to create an efficient dataset for each set.

    # Create a binary classification model, using a TextVectorization layer to preprocess each review. 
    # If the TextVectorization layer is not yet available (or if you like a challenge), try to create your own custom preprocessing layer: 
    # you can use the functions in the tf.strings package, for example lower() to make everything lowercase, regex_replace() to replace punctuation 
    # with spaces, and split() to split words on spaces. You should use a lookup table to output word indices, which must be prepared in the adapt() method.

    # Add an Embedding layer and compute the mean embedding for each review, multiplied by the square root of the number of words. 
    # This rescaled mean embedding can then be passed to the rest of your model.

    # Train the model and see what accuracy you get. Try to optimize your pipelines to make training as fast as possible.

    # Use TFDS to load the same dataset more easily: tfds.load("imdb_reviews").

## **Second Task :**

In [4]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import os

In [5]:
keras.backend.clear_session()
np.random.seed(33)
tf.random.set_seed(44)

In [6]:
# First Let's Download the dataset and complete the first step
    # Download the Large Movie Review Dataset, which contains 50,000 movies reviews from the Internet Movie Database. 
    # The data is organized in two directories, train and test, each containing a pos subdirectory with 12,500 positive reviews and 
    # a neg subdirectory with 12,500 negative reviews. Each review is stored in a separate text file. There are other files and folders 
    # (including preprocessed bag-of-words), but we will ignore them in this exercise.

from pathlib import Path

DOWNLOAD_ROOT = "http://ai.stanford.edu/~amaas/data/sentiment/"
FILENAME = "aclImdb_v1.tar.gz"
filepath = keras.utils.get_file(FILENAME, DOWNLOAD_ROOT + FILENAME, extract=True, cache_dir='/content/' )
path = Path(filepath).parent / "aclImdb"
path


# import tensorflow_datasets as tfds

# datasets = tfds.load(name="imdb_reviews")
# train_set, test_set = datasets["train"], datasets["test"]

PosixPath('/content/datasets/aclImdb')

In [8]:
# # Below Code is just for showing the Heirarchial Structure of the Root Folder of dataset
# # This can be analysed at OS level
for name, subdirs, files in os.walk(path):
    indent = len(Path(name).parts) - len(path.parts)
    print("    " * indent + Path(name).parts[-1] + os.sep)
    for index, filename in enumerate(sorted(files)):
        if index == 3:
            print("    " * (indent + 1) + "...")
            break
        print("    " * (indent + 1) + filename)

aclImdb/
    README
    imdb.vocab
    imdbEr.txt
    train/
        labeledBow.feat
        unsupBow.feat
        urls_neg.txt
        ...
        neg/
            0_3.txt
            10000_4.txt
            10001_4.txt
            ...
        pos/
            0_9.txt
            10000_8.txt
            10001_10.txt
            ...
        unsup/
            0_0.txt
            10000_0.txt
            10001_0.txt
            ...
    test/
        labeledBow.feat
        urls_neg.txt
        urls_pos.txt
        neg/
            0_2.txt
            10000_4.txt
            10001_1.txt
            ...
        pos/
            0_10.txt
            10000_7.txt
            10001_9.txt
            ...


In [24]:
path

PosixPath('/content/datasets/aclImdb')

In [9]:
# # Let's Find out the total reviews file for both training and testing data
def review_paths(dirpath):
    return [str(path) for path in dirpath.glob("*.txt")]

train_pos = review_paths(path / 'train' / 'pos')
train_neg = review_paths(path / 'train' / 'neg')
test_pos = review_paths(path / 'test' / 'pos')
test_neg = review_paths(path / 'test' / 'neg')

print("Training Positive Reviews Count : ", len(train_pos))
print("Training Negative Reviews Count : ", len(train_neg))
print("Testing Positive Reviews Count : ", len(test_pos))
print("Testing Negative Reviews Count : ", len(test_neg))


Training Positive Reviews Count :  12500
Training Negative Reviews Count :  12500
Testing Positive Reviews Count :  12500
Testing Negative Reviews Count :  12500


In [10]:
# Let's Split the test set into a validation set and a test set.
valid_pos = test_pos[:5000]
valid_neg = test_neg[:5000]
test_pos = test_pos[5000:]
test_neg = test_neg[5000:]

In [29]:
# Let's Print out the total Count of Training, Testing and Validation data set
print("Training Positive Reviews Count : ", len(train_pos))
print("Training Negative Reviews Count : ", len(train_neg))
print("Testing Positive Reviews Count : ", len(test_pos))
print("Testing Negative Reviews Count : ", len(test_neg))
print("Validation Postiive Reviews Count : ", len(valid_pos))
print("Validation Negative Reviews Count : ", len(valid_neg))

Training Positive Reviews Count :  12500
Training Negative Reviews Count :  12500
Testing Positive Reviews Count :  7500
Testing Negative Reviews Count :  7500
Validation Postiive Reviews Count :  5000
Validation Negative Reviews Count :  5000


In [11]:
# Let's Use tf.data to create an efficient dataset for each set.
# Since the dataset fits in memory, we can just load all the data using pure Python code and use tf.data.Dataset.from_tensor_slices():

def imdb_dataset(filepaths_positive, filepaths_negative):
    reviews = []
    labels = []
    for filepaths, label in ((filepaths_negative, 0), (filepaths_positive, 1)):
        for filepath in filepaths:
            with open(filepath) as review_file:
                reviews.append(review_file.read())
            labels.append(label)
    return tf.data.Dataset.from_tensor_slices(
        (tf.constant(reviews), tf.constant(labels))
        )


In [31]:
#Let's Print few of the reviews
for X, y in imdb_dataset(train_pos, train_neg).take(3):
    print(X)
    print(y)
    print()

tf.Tensor(b"Strange, almost all reviewers are highly positive about this movie. Is it because it's from 1975 and has Chamberlain and Curtis in it and therefore forgive the by times very bad acting and childish ways of storytelling? <br /><br />Maybe it's because some people get sentimental about this film because they have read the book? (I have not read the book, but I don't think that's a problem, film makers never presume that the viewers have read the book). <br /><br />Or is it because I am subconsciously irritated about the fact that English-speaking actors try to behave as their French counterparts?", shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int32)

tf.Tensor(b'For anyone who has seen and fallen in love with the stage musical A CHORUS LINE, the movie is a shoddy substitute. Not only are songs cut, but unnecessary plot twists added, new dance sequences choreographed, and, let\'s face it, Richard Attenborough just doesn\'t know how to film dancers.<br /><br />Onstage, 

In [12]:
# Let's make the dataset 
batch_size = 32

train_set = imdb_dataset(train_pos, train_neg).shuffle(25000).batch(batch_size).prefetch(1)
valid_set = imdb_dataset(valid_pos, valid_neg).batch(batch_size).prefetch(1)
test_set = imdb_dataset(test_pos, test_neg).batch(batch_size).prefetch(1)

Now Let's, Create a binary classification model, using a TextVectorization layer to preprocess each review. If the TextVectorization layer is not yet available (or if you like a challenge), try to create your own custom preprocessing layer: you can use the functions in the tf.strings package, for example lower() to make everything lowercase, regex_replace() to replace punctuation with spaces, and split() to split words on spaces. You should use a lookup table to output word indices, which must be prepared in the adapt() method._

Let's first write a function to preprocess the reviews, cropping them to 300 characters, converting them to lower case, then replacing <br /> and all non-letter characters to spaces, splitting the reviews into words, and finally padding or cropping each review so it ends up with exactly n_words tokens:

In [13]:
def preprocess(X_batch, n_words=50):
    shape = tf.shape(X_batch) * tf.constant([1, 0]) + tf.constant([0, n_words])
    Z = tf.strings.substr(X_batch, 0, 300)
    Z = tf.strings.lower(Z)
    Z = tf.strings.regex_replace(Z, b"<br\\s*/?>", b" ")
    Z = tf.strings.regex_replace(Z, b"[^a-z]", b" ")
    Z = tf.strings.split(Z)
    return Z.to_tensor(shape=shape, default_value=b"<pad>")

X_example = tf.constant(["It's a great, great movie! I loved it.", "It was terrible, run away!!!"])
preprocess(X_example)

<tf.Tensor: shape=(2, 50), dtype=string, numpy=
array([[b'it', b's', b'a', b'great', b'great', b'movie', b'i', b'loved',
        b'it', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>'],
       [b'it', b'was', b'terrible', b'run', b'away', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>',
        b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'<pad>', b'

Now let's write a second utility function that will take a data sample with the same format as the output of the preprocess() function, and will output the list of the top max_size most frequent words, ensuring that the padding token is first:

In [14]:
from collections import Counter

def get_vocabulary(data_sample, max_size=1000):
    preprocessed_reviews = preprocess(data_sample).numpy()
    counter = Counter()
    for words in preprocessed_reviews:
        for word in words:
            if word != b"<pad>":
                counter[word] += 1
    return [b"<pad>"] + [word for word, count in counter.most_common(max_size)]

get_vocabulary(X_example)

[b'<pad>',
 b'it',
 b'great',
 b's',
 b'a',
 b'movie',
 b'i',
 b'loved',
 b'was',
 b'terrible',
 b'run',
 b'away']

Now we are ready to create the TextVectorization layer. Its constructor just saves the hyperparameters (max_vocabulary_size and n_oov_buckets). The adapt() method computes the vocabulary using the get_vocabulary() function, then it builds a StaticVocabularyTable (see Chapter 16 for more details). The call() method preprocesses the reviews to get a padded list of words for each review, then it uses the StaticVocabularyTable to lookup the index of each word in the vocabulary:

In [15]:
class TextVectorization(keras.layers.Layer):
    def __init__(self, max_vocabulary_size=1000, n_oov_buckets=100, dtype=tf.string, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        self.max_vocabulary_size = max_vocabulary_size
        self.n_oov_buckets = n_oov_buckets

    def adapt(self, data_sample):
        self.vocab = get_vocabulary(data_sample, self.max_vocabulary_size)
        words = tf.constant(self.vocab)
        word_ids = tf.range(len(self.vocab), dtype=tf.int64)
        vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
        self.table = tf.lookup.StaticVocabularyTable(vocab_init, self.n_oov_buckets)
        
    def call(self, inputs):
        preprocessed_inputs = preprocess(inputs)
        return self.table.lookup(preprocessed_inputs)


In [16]:
# Let's try it on our small X_example we defined earlier:
text_vectorization = TextVectorization()

text_vectorization.adapt(X_example)
text_vectorization(X_example)

<tf.Tensor: shape=(2, 50), dtype=int64, numpy=
array([[ 1,  3,  4,  2,  2,  5,  6,  7,  1,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 1,  8,  9, 10, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0]])>

Looks good! As you can see, each review was cleaned up and tokenized, then each word was encoded as its index in the vocabulary (all the 0s correspond to the <pad> tokens).

Now let's create another TextVectorization layer and let's adapt it to the full IMDB training set (if the training set did not fit in RAM, we could just use a smaller sample of the training set by calling train_set.take(500)):

In [17]:
max_vocabulary_size = 100
n_oov_buckets = 100

sample_review_batches = train_set.map(lambda review, label: review)
sample_reviews = np.concatenate(list(sample_review_batches.as_numpy_iterator()),
                                axis=0)

text_vectorization = TextVectorization(max_vocabulary_size, n_oov_buckets,
                                       input_shape=[])
text_vectorization.adapt(sample_reviews)

In [18]:
# Let's run it on the same X_example, just to make sure the word IDs are larger now, since the vocabulary is bigger:

text_vectorization(X_example)

<tf.Tensor: shape=(2, 50), dtype=int64, numpy=
array([[  9,  14,   2,  64,  64,  12,   5, 139,   9,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  9,  13, 189, 177, 154,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])>

In [19]:
# Good! Now let's take a look at the first 10 words in the vocabulary:
text_vectorization.vocab[:10]



[b'<pad>', b'the', b'a', b'of', b'and', b'i', b'to', b'is', b'this', b'it']

These are the most common words in the reviews.

Now to build our model we will need to encode all these word IDs somehow. One approach is to create bags of words: for each review, and for each word in the vocabulary, we count the number of occurences of that word in the review. For example:

In [20]:
simple_example = tf.constant([[1, 3, 1, 0, 0], [2, 2, 0, 0, 0]])
tf.reduce_sum(tf.one_hot(simple_example, 4), axis=1)

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[2., 2., 0., 1.],
       [3., 0., 2., 0.]], dtype=float32)>

The first review has 2 times the word 0, 2 times the word 1, 0 times the word 2, and 1 time the word 3, so its bag-of-words representation is [2, 2, 0, 1]. Similarly, the second review has 3 times the word 0, 0 times the word 1, and so on. Let's wrap this logic in a small custom layer, and let's test it. We'll drop the counts for the word 0, since this corresponds to the <pad> token, which we don't care about.

In [21]:
class BagOfWords(keras.layers.Layer):
    def __init__(self, n_tokens, dtype=tf.int32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        self.n_tokens = n_tokens
    def call(self, inputs):
        one_hot = tf.one_hot(inputs, self.n_tokens)
        return tf.reduce_sum(one_hot, axis=1)[:, 1:]

In [22]:
# let's Test it
bag_of_words = BagOfWords(n_tokens=4)
bag_of_words(simple_example)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[2., 0., 1.],
       [0., 2., 0.]], dtype=float32)>

In [23]:
# It works fine! Now let's create another BagOfWord with the right vocabulary size for our training set:


n_tokens = max_vocabulary_size + n_oov_buckets + 1 # add 1 for <pad>
bag_of_words = BagOfWords(n_tokens)

We're ready to train the model!



In [24]:
model = keras.models.Sequential([
    text_vectorization,
    bag_of_words,
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid"),
])
model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit(train_set, epochs=5, validation_data=valid_set)

Epoch 1/5
782/782 [==============================] - 6s 6ms/step - loss: 0.6515 - accuracy: 0.6088 - val_loss: 0.6130 - val_accuracy: 0.6596
Epoch 2/5
782/782 [==============================] - 4s 5ms/step - loss: 0.5963 - accuracy: 0.6726 - val_loss: 0.6105 - val_accuracy: 0.6622
Epoch 3/5
782/782 [==============================] - 4s 5ms/step - loss: 0.5756 - accuracy: 0.6945 - val_loss: 0.6068 - val_accuracy: 0.6640
Epoch 4/5
782/782 [==============================] - 4s 5ms/step - loss: 0.5475 - accuracy: 0.7184 - val_loss: 0.6145 - val_accuracy: 0.6637
Epoch 5/5
782/782 [==============================] - 4s 5ms/step - loss: 0.5225 - accuracy: 0.7412 - val_loss: 0.6240 - val_accuracy: 0.6558


We get about 65% accuracy on the validation set after just the first epoch, but after that the model makes no significant progress. But it's okay as for now the point is just to perform efficient preprocessing using tf.data and Keras preprocessing layers.